### 3. Διάγραμμα Voronoi - Τριγωνοποίηση Delaunay

In [2]:
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import matplotlib.colors as mcolors
from copy import copy

def generate_points(n, seed=0):
    rng = np.random.default_rng(seed)
    points_set = set()
    while len(points_set) < n:
        point = tuple(rng.integers(0, 25, 2))
        points_set.add(point)
    return np.array(list(points_set))

points = generate_points(15, seed=100)
vor = Voronoi(points)
dlny = Delaunay(points)

# Generate a color map
n_colors = len(dlny.simplices)
colors = plt.cm.rainbow(np.linspace(0, 1, n_colors))
# colors[:, 3] = 0.5
np.random.shuffle(colors)

def circumcenter(simplex):
    """Calculate the circumcenter of a triangle."""
    pts = points[simplex]
    a, b, c = pts
    d = 2 * (a[0] * (b[1] - c[1]) + b[0] * (c[1] - a[1]) + c[0] * (a[1] - b[1]))
    ux = ((a[0]**2 + a[1]**2) * (b[1] - c[1]) + (b[0]**2 + b[1]**2) * (c[1] - a[1]) + (c[0]**2 + c[1]**2) * (a[1] - b[1])) / d
    uy = ((a[0]**2 + a[1]**2) * (c[0] - b[0]) + (b[0]**2 + b[1]**2) * (a[0] - c[0]) + (c[0]**2 + c[1]**2) * (b[0] - a[0])) / d
    return np.array([ux, uy])
circles = []
for i, simplex in enumerate(dlny.simplices):
    center = circumcenter(simplex)
    triangle = points[simplex]
    radius = np.linalg.norm(center - triangle[0])
    circles.append(plt.Circle(center, radius, fill=False, color=colors[i], linestyle='-',linewidth=1))

def plot_diagram(show_voronoi, show_delaunay, show_circumcircles):
    plt.close("all")
    # l = plt.cla()
    # plt.clf()
    fig, ax = plt.subplots(figsize=(10, 10))
    if show_voronoi:
        voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='black', line_width=1, line_alpha=0.8, point_size=0)
        for i, vertex in enumerate(vor.vertices):
            ax.plot(vertex[0], vertex[1], 'o', color='black', markerfacecolor=colors[i], markersize=5)
        

    if show_delaunay:
        for i, simplex in enumerate(dlny.simplices):
            triangle = points[simplex]
            ax.fill(triangle[:, 0], triangle[:, 1], color=colors[i]) 
            if not show_voronoi:
                centroid = np.mean(triangle, axis=0)
        ax.triplot(points[:, 0], points[:, 1], dlny.simplices, color='black', linewidth=1, alpha=0.8)

    if show_circumcircles:
        for i, simplex in enumerate(dlny.simplices):
            # if circles[i].figure is not None:
                # print("Removing circle")
                # plt.close(circles[i].figure)
                # circles[i].figure = None
            ax.add_patch(copy(circles[i]))
            # ax.plot(center[0], center[1], 'ro', markersize=4)  # Plot center in red
    # Plot the original points
    ax.plot(points[:, 0], points[:, 1], 'ko', markersize=6)

    # Set plot limits
    min_x = min(vor.vertices[:, 0].min(), points[:, 0].min())
    max_x = max(vor.vertices[:, 0].max(), points[:, 0].max())
    min_y = min(vor.vertices[:, 1].min(), points[:, 1].min())
    max_y = max(vor.vertices[:, 1].max(), points[:, 1].max())
    plt.xlim(min_x - 1.5, max_x + 1.5)
    plt.ylim(min_y - 1.5, max_y + 1.5)
    
    ax.set_title("Voronoi Diagram and Delaunay Triangulation")
    ax.set_aspect('equal')
    plt.show()

# Create checkbox widgets
voronoi_checkbox = widgets.Checkbox(value=False, description='Show Voronoi')
delaunay_checkbox = widgets.Checkbox(value=False, description='Show Delaunay')
circumcircles_checkbox = widgets.Checkbox(value=False, description='Show Circumcircles')
# Create an interactive output
interactive_output = widgets.interactive_output(
    plot_diagram, 
    {'show_voronoi': voronoi_checkbox,
     'show_delaunay': delaunay_checkbox,
     'show_circumcircles': circumcircles_checkbox}

)

# Display the widgets and the output
display(widgets.VBox([voronoi_checkbox, delaunay_checkbox, circumcircles_checkbox, interactive_output]),clear=True)


Στο παραπάνω γράφημα φαίνεται η σχέση δυΐσμου μεταξύ των διαγραμμάτων, επίσης φαίνονται και οι περιγρεγραμμένοι κενοι κύκλοι που ορίζουν τα τρίγων Delaunay. Τα χρώματα των τριγώνων αντιστοιχούν στις χρωματισμένες κορυφές Voronoi (η κορυφη μπορεί αν βρίσκεται εκτός τριγώνου, αυτό έχει αν κάνει με το σχήμα του, γι'αυτο τα διακρίνουμε απο το χρώμα). Οι αλγόριθμοι, αν πρόκειται για αυξητικούς έχουν μέση χρονική πολυπλοκότητα O(nlogn) και χείριστη O(n^2). Το n δηλώνει το πλήθος των σημείων, συνεπώς η αύξηση τους προκαλεί αύξηση στον χρόνο εκτέλεσης τους.